
### **Reddit Fetch**

We will be fetching Reddit post using official API :)

In [1]:
!pip install praw


In [2]:
import praw
import pandas as pd

reddit = praw.Reddit(
    client_id="KHbmxuI59KvPPjZOuPprvA",
    client_secret="GWtaNscUFjCWPmGJtg0NIaWVCfXSLw",
    user_agent="YOUR_APP_NAME / v1.0"
)

keyword = "Groww IPO"      # <--- change if needed
limit_posts = 50
limit_comments = 5

records = []

for submission in reddit.subreddit("all").search(keyword, limit=limit_posts):

    submission.comments.replace_more(limit=0)   # prevents loading "MoreComments"

    for i, comment in enumerate(submission.comments.list()):
        if i >= limit_comments:
            break

        records.append([
            submission.created_utc,
            submission.subreddit.display_name,
            submission.title,
            submission.selftext,
            submission.url,
            comment.body
        ])

df = pd.DataFrame(
    records,
    columns=["post_date_utc","subreddit","post_title","post_text","post_url","comment_text"]
)

df.to_csv("reddit_posts_with_comments.csv", index=False)
print(df)


     post_date_utc          subreddit  \
0     1.762269e+09       IndianStocks   
1     1.762269e+09       IndianStocks   
2     1.762269e+09       IndianStocks   
3     1.762269e+09       IndianStocks   
4     1.762269e+09       IndianStocks   
..             ...                ...   
229   1.756212e+09  IndianStockMarket   
230   1.756212e+09  IndianStockMarket   
231   1.761765e+09          IndianIPO   
232   1.761765e+09          IndianIPO   
233   1.761765e+09          IndianIPO   

                                            post_title  \
0        Is it a good idea to invest in the groww ipo?   
1        Is it a good idea to invest in the groww ipo?   
2        Is it a good idea to invest in the groww ipo?   
3        Is it a good idea to invest in the groww ipo?   
4        Is it a good idea to invest in the groww ipo?   
..                                                 ...   
229  Support from SBI and groww to release amount h...   
230  Support from SBI and groww to release 

### Google News

In [5]:
!pip install feedparser requests beautifulsoup4


In [6]:
import feedparser
from bs4 import BeautifulSoup
import urllib.parse

def fetch_google_news(ipo_name: str, limit: int = 20):
    query = urllib.parse.quote(ipo_name)
    rss_url = f"https://news.google.com/rss/search?q={query}&hl=en-IN&gl=IN&ceid=IN:en"

    feed = feedparser.parse(rss_url)

    results = []
    for entry in feed.entries[:limit]:
        # summary contains HTML → clean it
        soup = BeautifulSoup(entry.summary, "html.parser")
        clean_summary = soup.get_text(" ", strip=True)

        results.append({
            "title": entry.title,
            "link": entry.link,
            "summary": clean_summary,
            "published": entry.published
        })

    return results


# example:
news = fetch_google_news("Groww IPO", 20)

print(f"fetched {len(news)} articles")
for n in news[:3]:
    print("----")
    print(n["title"])
    print(n["summary"])


fetched 20 articles
----
Groww IPO day 3: GMP, subscription status, date to review. Apply or not? - livemint.com
Groww IPO day 3: GMP, subscription status, date to review. Apply or not? livemint.com Groww IPO Day 3 GMP Live: Check final day subscription status, price band, GMP, allotment date, valuation and more Moneycontrol At close, Groww IPO subscribed 18x The Times of India Groww IPO Day 3: Check Subscription Status & Key Highlights Groww Groww IPO oversubscribed 17.6x; QIB quota sees 22x bids Entrackr Highlights: Groww IPO subscribed over 17 times, but GMP cools to 5% – What’s the big worry? financialexpress.com Groww IPO closes today. Here's what the latest GMP signals India Today Groww IPO GMP Live Updates | GMP slips to 4% on Day 3; bidding ends with total subscription of over 17 ti... The Economic Times Groww IPO gets fully subscribed on Day 2 The Hindu
----
Lenskart, PhysicsWallah, and Groww: GMP trends suggest up to 22% listing gains for 9 IPOs next week - The Economic Times

Agent in Making

In [7]:
!pip -q install langchain langchain_groq "langgraph>=0.2.38" praw feedparser beautifulsoup4 pandas langchain-community


In [8]:
import os
from typing import TypedDict, List, Dict, Any, Optional
from datetime import datetime, timezone

import pandas as pd
import feedparser
from bs4 import BeautifulSoup
import praw

from langchain_groq import ChatGroq
from langchain.tools import tool
from langgraph.graph import StateGraph, END

# ---- LLM (expects GROQ_API_KEY in env) ----
# Set it here if you prefer (or export in your shell):
os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.2)


# Utility
def utc_iso(ts: float) -> str:
    try:
        return datetime.fromtimestamp(ts, tz=timezone.utc).isoformat()
    except Exception:
        return ""


In [9]:
# ---- Reddit Tool ----
# Inline credentials (as you requested)
REDDIT_CLIENT_ID     = "KHbmxuI59KvPPjZOuPprvA"
REDDIT_CLIENT_SECRET = "GWtaNscUFjCWPmGJtg0NIaWVCfXSLw"
REDDIT_USER_AGENT    = "YOUR_APP_NAME / v1.0"

def _make_reddit() -> praw.Reddit:
    return praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT,
    )

@tool("fetch_reddit_tool", return_direct=False)
def fetch_reddit_tool(keyword: str, limit_posts: int = 50, limit_comments: int = 5) -> List[Dict[str, Any]]:
    """Fetch Reddit posts and top-level comments for a keyword across r/all. Returns a list of dicts."""
    try:
        reddit = _make_reddit()
    except Exception as e:
        return [{"error": f"Reddit auth failed: {e}"}]

    records: List[Dict[str, Any]] = []

    try:
        for submission in reddit.subreddit("all").search(keyword, limit=limit_posts):
            try:
                submission.comments.replace_more(limit=0)
                # Collect up to limit_comments comments
                for i, comment in enumerate(submission.comments.list()):
                    if i >= limit_comments:
                        break
                    records.append({
                        "post_date_utc": utc_iso(submission.created_utc),
                        "subreddit": str(submission.subreddit.display_name),
                        "post_title": submission.title or "",
                        "post_text": submission.selftext or "",
                        "post_url": submission.url or "",
                        "comment_text": getattr(comment, "body", "") or "",
                    })
            except Exception:
                # Even if one submission fails, continue
                continue
    except Exception as e:
        return [{"error": f"Reddit search failed: {e}"}]

    return records


# ---- Google News Tool ----
@tool("fetch_google_news_tool", return_direct=False)
def fetch_google_news_tool(ipo_name: str, limit: int = 20) -> List[Dict[str, Any]]:
    """Fetch top Google News RSS results for an IPO name. Returns a list of dicts with title, link, summary, published."""
    import urllib.parse
    results: List[Dict[str, Any]] = []
    try:
        query = urllib.parse.quote(ipo_name)
        rss_url = f"https://news.google.com/rss/search?q={query}&hl=en-IN&gl=IN&ceid=IN:en"
        feed = feedparser.parse(rss_url)
        for entry in feed.entries[:limit]:
            soup = BeautifulSoup(getattr(entry, "summary", ""), "html.parser")
            clean_summary = soup.get_text(" ", strip=True)
            results.append({
                "title": getattr(entry, "title", ""),
                "link": getattr(entry, "link", ""),
                "summary": clean_summary,
                "published": getattr(entry, "published", ""),
            })
    except Exception as e:
        return [{"error": f"GNews fetch failed: {e}"}]
    return results


In [10]:
class AgentState(TypedDict, total=False):
    ipo_name: str
    reddit: List[Dict[str, Any]]
    news: List[Dict[str, Any]]
    analysis: str


def _truncate_text(text: str, max_chars: int) -> str:
    text = text or ""
    return (text[:max_chars] + "…") if len(text) > max_chars else text


def fetch_reddit_node(state: AgentState) -> AgentState:
    ipo = state["ipo_name"]
    # Use IPO keyword as provided + "IPO" helper if not present
    keyword = ipo if "ipo" in ipo.lower() else f"{ipo} IPO"
    reddit_data = fetch_reddit_tool.func(keyword=keyword, limit_posts=50, limit_comments=5)  # call underlying func
    return {**state, "reddit": reddit_data}


def fetch_news_node(state: AgentState) -> AgentState:
    ipo = state["ipo_name"]
    news_data = fetch_google_news_tool.func(ipo_name=ipo, limit=20)  # call underlying func
    return {**state, "news": news_data}


def _format_context_snippets(reddit: List[Dict[str, Any]], news: List[Dict[str, Any]], max_reddit: int = 30,
                             max_news: int = 12) -> str:
    # Reddit snippets
    r_lines: List[str] = []
    count = 0
    for r in reddit:
        if "error" in r:
            continue
        if count >= max_reddit:
            break
        title = _truncate_text(r.get("post_title", ""), 160)
        ctext = _truncate_text(r.get("comment_text", ""), 220)
        if not (title or ctext):
            continue
        r_lines.append(f"- [r/{r.get('subreddit', '')}] {title} || {ctext}")
        count += 1

    # News snippets
    n_lines: List[str] = []
    count = 0
    for n in news:
        if "error" in n:
            continue
        if count >= max_news:
            break
        title = _truncate_text(n.get("title", ""), 180)
        summ = _truncate_text(n.get("summary", ""), 260)
        n_lines.append(f"- {title} :: {summ}")

    ctx = []
    if r_lines:
        ctx.append("### Reddit (sample):\n" + "\n".join(r_lines))
    if n_lines:
        ctx.append("### News (sample):\n" + "\n".join(n_lines))
    return "\n\n".join(ctx)


def analysis_node(state: AgentState) -> AgentState:
    ipo = state["ipo_name"]
    reddit = state.get("reddit", []) or []
    news = state.get("news", []) or []

    context_snippets = _format_context_snippets(reddit, news)

    system_msg = (
        "You are a precise financial sentiment analyst. "
        "You analyze combined Reddit chatter and news headlines/summaries about a specific IPO. "
        "You MUST return output ONLY in the exact format:\n\n"
        "Sentiment Score: X.X/5\n"
        "Summary:\n"
        "- bullet...\n"
        "- bullet...\n\n"
        "Guidelines:\n"
        "- Score reflects near-term retail + media sentiment (0 = very negative, 5 = very positive).\n"
        "- Use short, concrete bullets (max 6) covering demand, risks, valuation vibes, red flags, catalysts.\n"
        "- DO NOT include sources, counts, or extra sections. No caveats. No emojis.\n"
        "- If data is sparse or mixed, reflect that in the score and bullets."
    )

    user_msg = (
        f"IPO: {ipo}\n\n"
        f"Here are sampled context snippets (Reddit + News). Use them to infer sentiment and produce the required output format.\n\n"
        f"{context_snippets if context_snippets else '[No snippets available]'}"
    )

    # LLM call
    resp = llm.invoke([("system", system_msg), ("user", user_msg)])
    analysis_text = resp.content if hasattr(resp, "content") else str(resp)

    return {**state, "analysis": analysis_text}


In [11]:
workflow = StateGraph(AgentState)
workflow.add_node("fetch_reddit", fetch_reddit_node)
workflow.add_node("fetch_news", fetch_news_node)
workflow.add_node("analyze", analysis_node)

workflow.set_entry_point("fetch_reddit")
workflow.add_edge("fetch_reddit", "fetch_news")
workflow.add_edge("fetch_news", "analyze")
workflow.add_edge("analyze", END)

app = workflow.compile()


In [12]:
def run_agent(ipo_name: str, save_csv: bool = False) -> Dict[str, Any]:
    """
    Runs the agent end-to-end and prints the final analysis in the exact requested format.
    Optionally saves fetched data to CSVs in the current directory.
    """
    result = app.invoke({"ipo_name": ipo_name})

    # Optional persistence for debugging
    if save_csv:
        try:
            if result.get("reddit"):
                pd.DataFrame(result["reddit"]).to_csv("reddit_posts_with_comments.csv", index=False)
            if result.get("news"):
                pd.DataFrame(result["news"]).to_csv("google_news_results.csv", index=False)
        except Exception as e:
            print(f"[warn] CSV save failed: {e}")

    print(result.get("analysis", "").strip())
    return result


In [13]:
# Ensure your GROQ_API_KEY is set before running:
# os.environ["GROQ_API_KEY"] = "YOUR_GROQ_KEY"

_ = run_agent("Groww IPO", save_csv=True)


Sentiment Score: 4.2/5
Summary:
- Strong demand for Groww IPO, subscribed 17.6x on Day 3.
- GMP softens to 11% reaching Rs 111, but still signals decent listing.
- Positive sentiment from investors, with many applying for shares.
- Some concerns about valuation and market conditions, but overall optimism.
- Long-term growth potential seen for the company.
- Focus shifts to allotment date, with steps to check status available.
